<a href="https://colab.research.google.com/github/thebirthoftropica-beep/portfolio/blob/main/proofreading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Unslothと依存関係のインストール

%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

print(f"{model_name} のロード完了")

In [ ]:
# 2. モデル（Llama-3-ELYZA-JP-8b）のロード
from unsloth import FastLanguageModel
import torch

# 扱うモデルの名前
model_name = "elyza/Llama-3-ELYZA-JP-8b"

# モデルとトークナイザーを読み込む
# 4bit量子化（max_seq_length = 2048, load_in_4bit = True）でメモリ節約
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print(f"{model_name} のロード完了")

In [ ]:
import pandas as pd
from datasets import Dataset

# 1. プロンプトの「型（テンプレート）」を作る
alpaca_prompt = """下記は、新聞記者ハンドブックに基づく校閲の指示と、対象の文章です。指示に従って適切に修正された文章を出力してください。

### 指示:
{}

### 入力:
{}

### 出力:
{}"""

# EOS_TOKEN（文の終わりを教える記号）を準備
EOS_TOKEN = tokenizer.eos_token

# 2. CSVデータを読み込んで、プロンプトに流し込む関数
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # テンプレートに中身を埋め込む
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 3. CSVファイルを読み込む
df = pd.read_csv("data.csv", encoding='shift_jis')

# データセット形式に変換
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True)

# 4. ちゃんと読み込めたか、最初の1件を表示して確認
print("\n--- データの確認（最初の1件） ---")
print(dataset[0]["text"])
print("---------------------------------")

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 1. LoRAアダプタの設定（PEFT）
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # AIの更新するパラメータ数
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# 2. 学習の設定（ハイパーパラメータ）
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # 学習回数
        learning_rate = 2e-4, # 学習率
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# 3. 学習の実行
print("学習スタート！")
trainer_stats = trainer.train()
print("学習完了！")

In [ ]:
# 1. 推論モードに切り替え
FastLanguageModel.for_inference(model)

# ==========================================
# ここにテストしたい「ダメな文章」を入れる
# ==========================================
input_text = "雪辱を晴らす。"

prompt = alpaca_prompt.format(
    "新聞記者ハンドブックに基づいて校閲して。", # 指示（学習データと合わせる）
    input_text, # 入力
    "", # 出力（ここは空欄にしてAIに続きを書かせる）
)

# 3. AIに読ませる準備（トークナイズ）
inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

# 4. 推論の実行（生成テスト）
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

# 5. 結果をデコードして表示
# 余計な記号を消して読みやすくする
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("\n========== 校閲結果 ==========\n")
# 結果全体を表示（プロンプト含む）
print(result[0])
print("\n==============================")

In [ ]:
# 1. 学習したLoRAアダプタ（差分データ）を保存
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

# 2. フォルダをzipに圧縮（ダウンロードしやすくするため）
!zip -r lora_model.zip lora_model

# 3. PCにダウンロード
from google.colab import files
files.download('lora_model.zip')